## Trains an LSTM model

#### inputs:
- df_train01.csv
- df_test01.csv
- df_chunk_train01.csv
- df_chunk_test01.csv

#### outputs:
- normalizer.pkl
- batcher_train_lite_model03.pkl
- model03_epoch199.pkl

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pickle
import sys
from importlib import reload
import logging

%matplotlib inline

In [14]:
sys.path.append('..')

#### Load data

In [15]:
df_train = pd.read_csv('../data/df_train02.csv',  parse_dates=['Datetime'])


df_chunk_train = pd.read_csv('../data/df_chunk_train02.csv')


In [29]:
df_train.head()

,Unnamed: 0,LocId,Datetime,Moisture,Latitude,Longitude,ChunkBegin,ChunkId,precip_intensity,precip_probability,...,wind_gust,cloud_cover,visibility,Distance,Eawc,IsTmpNull,Day,Hour,Month,Weekday
0,647,6,2018-11-26 14:00:00,0.424321,47.594,19.362,True,2,0.040627,0.81,...,0.557199,1.0,0.115889,1.962063,169.59,False,0.833333,0.608696,0.909091,0.0
1,648,6,2018-11-26 15:00:00,0.442199,47.594,19.362,False,2,0.041391,0.79,...,0.597072,1.0,0.193065,1.962063,169.59,False,0.833333,0.652174,0.909091,0.0
2,649,6,2018-11-26 16:00:00,0.446845,47.594,19.362,False,2,0.034764,0.76,...,0.623144,1.0,0.239918,1.962063,169.59,False,0.833333,0.695652,0.909091,0.0
3,650,6,2018-11-26 17:00:00,0.437385,47.594,19.362,False,2,0.025321,0.72,...,0.638479,1.0,0.294476,1.962063,169.59,False,0.833333,0.739130,0.909091,0.0
4,651,6,2018-11-26 18:00:00,0.425821,47.594,19.362,False,2,0.018540,0.63,...,0.143128,1.0,0.920089,1.962063,169.59,False,0.833333,0.782609,0.909091,0.0


In [16]:
df_test = pd.read_csv('../data/df_test02.csv',  parse_dates=['Datetime'])
df_chunk_test = pd.read_csv('../data/df_chunk_test02.csv')

#### Create a normalizer object

In [17]:
from lib import normalizing as nr

In [18]:
normalizer = nr.Normalizer(meanstd=['temperature', 'apparent_temperature', 'dew_point',
                                   'wind_speed', 'wind_gust'],
                      minmax=['visibility', 'precip_intensity', 'Moisture',
                              'Day', 'Hour', 'Month', 'Weekday'])

In [19]:
df_train = normalizer.normalize(df_train, init=True)

In [20]:
normalizer.save('../data/normalizer2.pkl')

#### Create batcher object

In [21]:
from lib import batching as bt

In [22]:
bt = reload(bt)

In [23]:
col_feats_prev = ['humidity', 'precip_probability',
                  'precip_intensity', 'temperature', 'Moisture']
col_feats_curr = ['precip_probability', 'humidity', 'cloud_cover',
                  'apparent_temperature', 'dew_point', 'wind_speed',
                  'wind_gust', 'visibility', 'precip_intensity',
                  'Day', 'Hour', 'Month', 'Weekday'
                 ]

In [24]:
batcher_train = bt.BatcherTrain(df_train,
                df_chunk_train,
                40,
                36,
                col_dt='Datetime', # Datetime column name
                 col_val='Moisture', # ts column name to be predicted
                 col_gr = 'ChunkId',
                 col_feats_prev = col_feats_prev, # prev-feature column names
                 col_feats_current = col_feats_curr, # current feature column names
                 minlen=25, # chunk lengths are randomly sampled, minimum chunk length
                 maxlen=240        
               )

Save batcher without data for passing parameters to tester at v07.test01.ipynb

In [25]:
df_ts = batcher_train.df_ts
batcher_train.df_ts = None
batcher_train.save('../data/batcher_train_lite_eawc_model03.pkl')
batcher_train.df_ts = df_ts

#### Create LSTM model

In [26]:
from lib import lstm

In [27]:
lstm = reload(lstm)

In [28]:
model = lstm.LSTM(
    batcher_train.n_feat,
    200,
    hiddens_before=[800],
    hiddens_after=[800, batcher_train.out_time_len],
    lr_init=0.001,
    use_gpu=True
)

RuntimeError: cuda runtime error (100) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:47

In [46]:
model.move_cpu()

Train the model

In [ ]:
model.train_model(batcher_train, max_epochs=200, batchsize=150, save_every=None, save_folder=None)

In [15]:
import torch

In [ ]:
torch.save(model, '../data/model03_epoch199.pkl')